In [1]:
#  _____ __  __ _   _           _   _ _   _
# |  ___|  \/  | | | |         | \ | | \ | |
# | |_  | |\/| | | | |  _____  |  \| |  \| |
# |  _| | |  | | |_| | |_____| | |\  | |\  |
# |_|   |_|  |_|\___/          |_| \_|_| \_|
#
"""
 @authors: Matteo Larcher
"""


# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# import libraries
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import os

# custom libraries
from FMU_layer import *

# Load the TensorBoard notebook extension
%load_ext tensorboard

2024-10-31 14:49:53.398852: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# Configs
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# set keras float precision
tf.keras.backend.set_floatx('float64')

# set eager execution
tf.config.run_functions_eagerly(False)

# set random seeds
tf.keras.utils.set_random_seed(123)

# set the path of the FMU model
dirname = os.getcwd()
fmu_path = os.path.join(dirname, "fmu_model", "xy_model_om_dd.fmu")

# simulation parameters
start_time = 0.0
stop_time = 20.0
step_size = 0.1
t_vect = np.arange(start_time, stop_time, step_size)

In [3]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# Test FMU Layer
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

start_values = [0.0, 0.0]
parameters = [1.0]
input_x = np.cos(t_vect).astype(tf.keras.backend.floatx()).reshape(1, -1)
input_y = np.sin(t_vect).astype(tf.keras.backend.floatx()).reshape(1, -1)
input_data = tf.Variable(tf.stack([input_x, input_y], axis=2), dtype=tf.keras.backend.floatx(), trainable=True)

fmu_layer = FMULayer(
    fmu_path,
    start_time,
    start_values,
    parameters,
    step_size,
    return_sequences=True,
    return_state=False,
    stateful=True,
    name="test_fmu_layer",
    do_step_in_gradient=True,
)

output = fmu_layer(input_data)

# Plot the results
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=t_vect, y=input_x[0], name="Input x", mode="lines"), row=1, col=1)
fig.add_trace(go.Scatter(x=t_vect, y=input_y[0], name="Input y", mode="lines"), row=1, col=1)
fig.add_trace(go.Scatter(x=t_vect, y=output[0][:,0], name="z", mode="lines"), row=2, col=1)
fig.update_layout(title="FMU Layer Test", xaxis_title="Time [s]", yaxis_title="Value")
fig.show()


In [4]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# Test the layer gradient
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

with tf.GradientTape() as tape:
    tape.watch(input_data)
    outputs = fmu_layer(input_data)
    # gather gradients
    gradients = tape.gradient(outputs, input_data)

# Plot the gradients
fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(x=t_vect, y=outputs[0][:,0], name="z", mode="lines"), row=1, col=1)
fig.add_trace(go.Scatter(x=t_vect, y=input_x[0], name="x", mode="lines"), row=1, col=1)
fig.add_trace(go.Scatter(x=t_vect, y=input_y[0], name="y", mode="lines"), row=1, col=1)
fig.add_trace(go.Scatter(x=t_vect, y=gradients[0][:,0], name="dz/dx", mode="lines", line=dict(dash='dot')), row=1, col=1)
fig.add_trace(go.Scatter(x=t_vect, y=gradients[0][:,1], name="dz/dy", mode="lines", line=dict(dash='dot')), row=1, col=1)

fig.update_layout(title="FMU Layer Gradient Test", xaxis_title="Time [s]", yaxis_title="Value")
fig.show()


In [5]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# Model definition
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

def get_model(fmu_path, start_time, start_values, parameters, step_size):
    x_in = tf.keras.layers.Input(shape=(None, 1), batch_size=1, name="x_in")
    y_in = tf.keras.layers.Input(shape=(None, 1), batch_size=1, name="y_in")
    inputs = tf.keras.layers.Concatenate(axis=2, name="inputs")([x_in, y_in])

    # Add a dense layer before the FMULayer
    dense_1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(2), name="dense_before")(inputs)

    z_out = FMULayer(
        fmu_path,
        start_time,
        start_values,
        parameters,
        step_size,
        return_sequences=True,
        return_state=False,
        stateful=True,
        name="xy_fmu",
        do_step_in_gradient=False,
    )(dense_1)

    # custom layer that performs (in1*in2)+parameters[0]
    # z_out = tf.keras.layers.Lambda(lambda x: x[:,:,0]*x[:,:,1]+1.0, name="custom_layer", output_shape=(None,1))(dense_1)

    # Add a dense layer after the FMULayer
    dense_2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1), name="dense_after")(z_out)

    model = tf.keras.Model(inputs=[x_in, y_in], outputs=dense_2)
    return model

# create the model with the FMU layer
model = get_model(fmu_path, start_time, [0.0, 0.0], [2.0], step_size)

model.summary()

# compile the model
model.compile(optimizer=tf.keras.optimizers.Nadam(learning_rate=0.1), loss='mse')

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ x_in (InputLayer)   │ (1, None, 1)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ y_in (InputLayer)   │ (1, None, 1)      │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inputs              │ (1, None, 2)      │          0 │ x_in[0][0],       │
│ (Concatenate)       │                   │            │ y_in[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_before        │ (1, None, 2)      │          6 │ inputs[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ fmu_layer_1         │ (1, None, 1)      │          0 │ dense_before[0][… │
│ (FMULayer)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_after         │ (1, None, 1)      │          2 │ fmu_layer_1[0][0] │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8 (64.00 B)

 Trainable params: 8 (64.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
# create the input data
input_x = np.cos(t_vect).reshape(1, -1, 1)
input_y = np.sin(t_vect).reshape(1, -1, 1)
input_data = [input_x, input_y]

In [7]:
# predict
z = model.predict(input_data)
# target
target = 4*((2*input_x+4*input_y+1) * (3*input_y-2*input_x-3)) + 2

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


In [8]:
# Plot the results
fig = go.Figure()

# Add traces for z and target
fig.add_trace(go.Scatter(x=t_vect, y=z.reshape(-1), mode='lines', name='z'))
fig.add_trace(go.Scatter(x=t_vect, y=target[0, :, 0], mode='lines', name='target', line=dict(dash='dash')))

# Update layout
fig.update_layout(
    title="Model Prediction vs Target",
    xaxis_title="Time [s]",
    yaxis_title="z",
    legend=dict(x=0, y=1, traceorder="normal"),
)

# Show the figure
fig.show()

In [9]:
# TensorBoard callback for live plotting
log_dir = os.path.join(dirname, "logs", "fit", "xy_model")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=10)

# custom TensorBoard callback
class CustomTensorBoard(tf.keras.callbacks.TensorBoard):
    def __init__(self, *args, **kwargs):
        super(CustomTensorBoard, self).__init__(*args, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 == 0:  # Generate images every 5 epochs
            # plot model inputs, outputs and target
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=t_vect, y=model.predict(input_data).reshape(-1), mode='lines', name='z'))
            fig.add_trace(go.Scatter(x=t_vect, y=target[0, :, 0], mode='lines', name='target', line=dict(dash='dash')))
            fig.update_layout(title="Model Prediction vs Target", xaxis_title="Time [s]", yaxis_title="z")

            # add the plot to TensorBoard
            writer = tf.summary.create_file_writer(log_dir)
            with writer.as_default():
                img = tf.image.decode_image(fig.to_image(format="png"), channels=4)
                img = tf.expand_dims(img, 0)  # add batch dimension
                tf.summary.image(name='model', data=img, step=epoch)

        # call the original on_epoch_end method
        super(CustomTensorBoard, self).on_epoch_end(epoch, logs)

# Set the TensorBoard port
os.environ['TENSORBOARD_PORT'] = '6006'

In [15]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# fit the model
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)
history = model.fit(input_data, target.reshape(1, -1, 1), epochs=100, batch_size=1, shuffle=False, callbacks=[early_stopping, tensorboard_callback, CustomTensorBoard()])


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/steploss: 0.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 0.0080
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step - loss: 0.0076
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 416ms/step - loss: 0.0071
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - loss: 0.0067
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - loss: 0.0063
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 0.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 638ms/step - loss: 0.0060
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - loss: 0.0056
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - loss: 0.0053
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 269ms/step - loss: 0.0050
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step - loss: 0.0047
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 0.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 601ms/step - loss: 0.0044
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step - loss: 0.0042
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step - loss: 0.0039
E

In [16]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# Plot the training history
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

fig = go.Figure()
fig.add_trace(go.Scatter(x=history.epoch, y=history.history['loss'], mode='lines', name='loss'))
fig.update_layout(title="Training History", xaxis_title="Epoch", yaxis_title="Loss")
fig.show()

In [17]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# Predict and plot the results of the trained model
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

z = model.predict(input_data)

# Create a figure
fig = go.Figure()

# Add traces for z and target
fig.add_trace(go.Scatter(x=t_vect, y=z.reshape(-1), mode='lines', name='z'))
fig.add_trace(go.Scatter(x=t_vect, y=target[0, :, 0], mode='lines', name='target', line=dict(dash='dash')))

# Update layout
fig.update_layout(
    title="Model Prediction vs Target",
    xaxis_title="Time [s]",
    yaxis_title="z",
    legend=dict(x=0, y=1, traceorder="normal"),
)

# Show the figure
fig.show()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step


In [18]:
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# Export the model to ONNX
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

import tf2onnx
save_file = os.path.join(dirname, "onnx", "xy_model")

# Convert the Keras model layers to ONNX


AttributeError: 'Functional' object has no attribute '_get_save_spec'